In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from cv2 import imread, createCLAHE # read and equalize images
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
from __future__ import print_function, division

# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# image imports
from skimage import io, transform
from PIL import Image

# general imports
import os
import time
from shutil import copyfile
from shutil import rmtree

# data science imports
import pandas as pd
import numpy as np
import csv

In [ ]:
from itertools import chain
import random
np.random.seed(25)

df = pd.read_csv('../input/data/Data_Entry_2017.csv')
df.drop(['OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'], axis=1, inplace=True)
num_obs = len(df)
# print('Number of observations:',num_obs)

my_glob = glob('../input/data/images*/images/*.png')
# print('Number of Observations: ', len(my_glob))

full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

train_val_list = pd.read_csv('../input/data/train_val_list.txt', header=None, names = ['image_list'])
test_list = pd.read_csv('../input/data/test_list.txt', header=None, names = ['image_list'])

train = df[df['Image Index'].isin(train_val_list['image_list'].values)].reset_index(drop=True)
test = df[df['Image Index'].isin(test_list['image_list'].values)].reset_index(drop=True)

labels_discard = ['Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Hernia', 'Pleural_Thickening']
for i in labels_discard:
    train = train[~train['Finding Labels'].str.contains(i)]
    test = test[~test['Finding Labels'].str.contains(i)]

# train = pd.concat([train[~train['Finding Labels'].str.contains('No Finding')],
#                   train[train['Finding Labels'].str.contains('No Finding')].drop_duplicates(subset=['Finding Labels', 'Patient ID', 'View Position'], keep='first')]).sort_values(by='Image Index')

def one_hot_enc(df):
    df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
    all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    for c_label in all_labels:
        if len(c_label)>1: # leave out empty labels
            df[c_label] = df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
    return df
        
train = one_hot_enc(train)
test = one_hot_enc(test)

train_image_paths = list(flatten(train['full_path'].values))
test_image_paths = list(flatten(test['full_path'].values))

l = np.unique(train['Patient ID'].values)
np.random.shuffle(l)
cut = int(np.round(((90/100)*len(l)), decimals=0))
train_values = l[:cut]
val_values = l[cut:]
train_dict = dict.fromkeys(train_values, 'train')
train_dict.update(dict.fromkeys(val_values, 'val'))
train['fold'] = train['Patient ID'].map(train_dict.get)

train = train[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
               'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
test['fold'] = 'test'
test = test[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
             'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]

In [ ]:
healthy = train[train[['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']].sum(axis=1) == 0]
healthy.reset_index(drop=True, inplace=True)
healthy.shape

In [ ]:
healthy[:500]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch.optim as optim
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F

torch.manual_seed(98)

In [ ]:
im1 = cv2.imread(healthy['full_path'].iloc[0])
# im1 = cv2.imread("black-line-md.png",0)
im1 = cv2.normalize(im1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

In [ ]:
# Assuming all images are the same size, get dimensions of first image
w,h=Image.open(healthy['full_path'].iloc[0]).size
N=len(healthy[:2000])

# Create a numpy array of floats to store the average (assume RGB images)
arr=np.zeros((h,w,3),np.float)

# Build up average pixel intensities, casting each image as an array of floats
for i in range(N):
    im = Image.open(healthy['full_path'].iloc[i])
    im = im.convert('RGB')
    imarr=np.array(im,dtype=np.float)
    arr=arr+imarr/N

# Round values in array and cast as 8-bit integer
arr=np.array(np.round(arr),dtype=np.uint8)

# Generate, save and preview final image
out=Image.fromarray(arr,mode="RGB")

dim = (256, 256)
out = cv2.resize(np.asarray(out), dim)

# img = Image.fromarray(out, 'RGB')
img = out
w, h = 224, 224
center = img.shape
x = center[1]/2 - w/2
y = center[0]/2 - h/2

anchor = img[int(y):int(y+h), int(x):int(x+w)]
# anchor = cv2.resize(anchor, (224,224))
# anchor = cv2.normalize(anchor, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

plt.figure()
f, axarr = plt.subplots(1,2) 

axarr[0].imshow(out)
axarr[1].imshow(anchor)
plt.show()

In [ ]:
anchor.shape

In [ ]:
from PIL import Image

class CreateDataset(Dataset):
    
    def __init__(self, train, fold, anchor, transform=None):
        self.df = train
        self.transform = transform
        self.df = self.df[self.df['fold'] == fold]
        
        self.df = self.df.set_index("Image Index")
        self.PRED_LABEL = ['Atelectasis',
                           'Cardiomegaly',
                           'Effusion',
                           'Infiltration',
                           'Mass',
                           'Nodule',
                           'Pneumonia',
                           'Pneumothorax']
        self.anchor = anchor
        RESULT_PATH = "results/"
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        image = Image.open(self.df.iloc[idx, 0])
        image = image.convert('RGB')
        anchor = self.anchor
        anchor = anchor.convert('RGB')
        
        label = np.zeros(len(self.PRED_LABEL), dtype=int)
        for i in range(0, len(self.PRED_LABEL)):
            if(self.df[self.PRED_LABEL[i].strip()].iloc[idx].astype('int') > 0):
                label[i] = self.df[self.PRED_LABEL[i].strip()
                                  ].iloc[idx].astype('int')

        if self.transform:
            image = self.transform(image)
            anchor = self.transform(anchor)
        
        return image, anchor

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
N_LABELS = 8  # we are predicting 14 labels
data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomHorizontalFlip(),
        transforms.Resize(224),
        # because resize doesn't always give 224 x 224, this ensures 224 x
        # 224
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

anchor_temp = Image.fromarray(anchor)
# anchor_temp = anchor

transformed_datasets = {}
transformed_datasets['train'] = CreateDataset(healthy, 'train', anchor_temp, transform=data_transforms['test'])
transformed_datasets['val'] = CreateDataset(train, 'val', anchor_temp, transform=data_transforms['test'])

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(
    transformed_datasets['train'],
    batch_size=16,
    shuffle=True,
    num_workers=8,
    pin_memory=True)
dataloaders['val'] = torch.utils.data.DataLoader(
    transformed_datasets['val'],
    batch_size=16,
    shuffle=True,
    num_workers=8,
    pin_memory=True)

In [ ]:
import torch
from PIL import Image
from torch.autograd import Variable


def load_image(filename, size=None, scale=None):
    img = Image.open(filename)
    if size is not None:
        img = img.resize((size, size), Image.ANTIALIAS)
    elif scale is not None:
        img = img.resize((int(img.size[0] / scale), int(img.size[1] / scale)), Image.ANTIALIAS)
    return img


def save_image(filename, data):
    img = data.clone().clamp(0, 255).numpy()
    img = img.transpose(1, 2, 0).astype("uint8")
    img = Image.fromarray(img)
    img.save(filename)


def gram_matrix(y):
    (b, ch, h, w) = y.size()
    features = y.view(b, ch, w * h)
    features_t = features.transpose(1, 2)
    gram = features.bmm(features_t) / (ch * h * w)
    return gram


def normalize_batch(batch):
    # normalize using imagenet mean and std
    mean = batch.data.new(batch.data.size())
    std = batch.data.new(batch.data.size())
    mean[:, 0, :, :] = 0.485
    mean[:, 1, :, :] = 0.456
    mean[:, 2, :, :] = 0.406
    std[:, 0, :, :] = 0.229
    std[:, 1, :, :] = 0.224
    std[:, 2, :, :] = 0.225
    batch = torch.div(batch, 255.0)
    batch -= Variable(mean)
    batch = batch / Variable(std)
    return batch

In [ ]:
# the vgg16 is borrowed from the pytorch tutorial
# the resent is used to reimplemented and the comparision experiments
from collections import namedtuple
import os
import torch
from torchvision import models

class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        
        resnet50_pretrained = models.resnet50()
        num_ftrs = resnet50_pretrained.fc.in_features
        resnet50_pretrained.fc = nn.Sequential(
            nn.Linear(num_ftrs, 8), nn.Sigmoid())

        resnet50_pretrained.load_state_dict(torch.load('/kaggle/input/models-for-localisation-testing/resnet50_5_chestxray8.pth'))
        resnet50_pretrained.eval()
        
        self.model  = list(resnet50_pretrained.children())
        
        self.noslice1 = self.model[0]
        self.noslice2 = self.model[1]
        self.noslice3 = self.model[2]
        self.noslice4 = self.model[3]
        
        self.slice1 = self.model[4]
        self.slice2 = self.model[5]
        self.slice3 = self.model[6]
        self.slice4 = self.model[7]
        
        
    def forward(self, x):
#         x = x.to(device)
        o = x
#         for i in range(4):
#             o = self.model[i](o)
            
        o = self.noslice1(o)
        o = self.noslice2(o)
        o = self.noslice3(o)
        o = self.noslice4(o)

        o = self.slice1(o)
        feature1 = o
        o = self.slice2(o)
        feature2 = o
        o = self.slice3(o)
        feature3 = o
        o = self.slice4(o)
        feature4 = o
#         o = self.fc(o)
#         transform_param = o
        
#         return feature1, feature2, feature3, feature4
        return feature1, feature2

In [ ]:
from collections import namedtuple
import os
import torch
from torchvision import models
import torch.nn.functional as F
torch.manual_seed(98)

class AlignmentNetwork(torch.nn.Module):
    def __init__(self):
        super(AlignmentNetwork, self).__init__()
        
        resnet18 = models.resnet18(pretrained=True)
        num_ftrs = resnet18.fc.in_features
        resnet18.fc = nn.Linear(num_ftrs, 6)
        
        self.model  = resnet18
        self.model.fc.weight.data.zero_()
        self.model.fc.bias.data.copy_(torch.tensor([1,0,0,0,1,0]))
        
        
    def forward(self, x):
        theta = self.model(x)
        theta = theta.view(-1, 2, 3).squeeze()
        
        self.grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, self.grid)
        m = nn.AvgPool2d(16, stride=16)
        x = m(x)
        x = F.interpolate(x, (224,224), mode='bilinear')
        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

alignment_network = AlignmentNetwork()
alignment_network = alignment_network.to(device)
alignment_network.train()

extract_model = ResNet50()
extract_model = extract_model.to(device)
extract_model = extract_model.type(torch.cuda.FloatTensor)
# extract_model.eval()

criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(alignment_network.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
# optimizer = optim.Adam(alignment_network.parameters(),lr=0.001, betas=(0.9, 0.999))
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 40)
loss_val_list = []

print("Starting Training")
for epoch in range(5):
    t0 = time.time()
    total_loss = 0
    for batch in dataloaders['train']:
        perceptual_loss = 0
        euclidean_loss = 0
        x0, anchor = batch
        x0, anchor = x0.to(device), anchor.to(device)
        transformed_z0 = alignment_network(x0)
#         transformed_z0 = transform_input(z0, x0)
#         transformed_z0 = transformed_z0.to(device)

        features_anchor = extract_model(anchor)
        features_transformed = extract_model(transformed_z0)
                                                                                                                                                                                                     
        optimizer.zero_grad()
        for i in range(len(features_anchor)):
            temp_loss = criterion(features_transformed[i], features_anchor[i])
            perceptual_loss += temp_loss
        
        euclidean_loss = criterion(transformed_z0, anchor)
        
        loss = euclidean_loss + perceptual_loss

        total_loss += loss
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(dataloaders['train'])
    torch.save(alignment_network.state_dict(), f'2feat_no_crop_fixed_pt_alignment_network_{epoch+1}.pth')
    print(f"epoch: {epoch+1:>02}, loss: {avg_loss:.5f}. Time taken: {((time.time()-t0)/60):.3f} mins")
    loss_val_list.append(avg_loss)

In [ ]:
features_transformed[0].shape

In [ ]:
plt.imshow(features_transformed[0][0][0].cpu().detach().numpy())

In [ ]:
from collections import namedtuple
import os
import torch
from torchvision import models
import torch.nn.functional as F
torch.manual_seed(98)

class AlignmentNetwork_Val(torch.nn.Module):
    def __init__(self):
        super(AlignmentNetwork_Val, self).__init__()
        
        resnet18 = models.resnet18()
        num_ftrs = resnet18.fc.in_features
        resnet18.fc = nn.Linear(num_ftrs, 6)
        
        self.model  = resnet18
        
        
    def forward(self, x):
        theta = self.model(x)
        theta = theta.view(-1, 2, 3).squeeze()
        
        self.grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, self.grid)
#         m = nn.AvgPool2d(16, stride=16)
#         x = m(x)
#         x = F.interpolate(x, (224,224), mode='bilinear')
        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

alignment_network = AlignmentNetwork_Val()
alignment_network.load_state_dict(torch.load('/kaggle/working/2feat_no_crop_fixed_pt_alignment_network_5.pth'))
alignment_network = alignment_network.to(device)
alignment_network.eval()

extract_model = ResNet50()
extract_model = extract_model.to(device)
extract_model = extract_model.type(torch.cuda.FloatTensor)
# extract_model.eval()

criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(alignment_network.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
# optimizer = optim.Adam(alignment_network.parameters(),lr=0.001, betas=(0.9, 0.999))
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 40)
loss_val_list = []

print("Starting Training")
for epoch in range(5):
    t0 = time.time()
    total_loss = 0
    for batch in dataloaders['val']:
        perceptual_loss = 0
        euclidean_loss = 0
        x0, anchor = batch
        x0, anchor = x0.to(device), anchor.to(device)
        transformed_z0 = alignment_network(x0)
#         transformed_z0 = transform_input(z0, x0)
#         transformed_z0 = transformed_z0.to(device)

#         features_anchor = extract_model(anchor)
#         features_transformed = extract_model(transformed_z0)
                                                                                                                                                                                                     
#         optimizer.zero_grad()
#         for i in range(len(features_anchor)):
#             temp_loss = criterion(features_transformed[i], features_anchor[i])
#             perceptual_loss += temp_loss
        
#         euclidean_loss = criterion(transformed_z0, anchor)
        
#         loss = euclidean_loss + perceptual_loss

#         total_loss += loss
#         loss.backward()
#         optimizer.step()
        
        break
    break
    
#     avg_loss = total_loss / len(dataloaders['train'])
#     torch.save(alignment_network.state_dict(), f'fixed_pt_alignment_network_{epoch+1}.pth')
#     print(f"epoch: {epoch+1:>02}, loss: {avg_loss:.5f}. Time taken: {((time.time()-t0)/60):.3f} mins")
#     loss_val_list.append(avg_loss)

In [ ]:
plt.imshow(np.moveaxis(x0[15].cpu().detach().numpy(), 0, 2))
plt.axis('off')
plt.show()
plt.savefig('1.png')

In [ ]:
plt.imshow(np.moveaxis(transformed_z0[15].cpu().detach().numpy(), 0, 2))
plt.axis('off')
plt.show()
plt.savefig('2.png')

In [ ]:
transformed_z0.shape

In [ ]:
x0.shape

In [ ]:
plt.imshow(np.moveaxis(x0[0].cpu().detach().numpy(), 0, 2))